### Diffing Payloads

#### Standard Library's Difflib

> * how do we compare two JSON documents?
> * one (not so great) idea: string compare them using the built-in difflib

In [6]:
import difflib

In [7]:
text1 = "apple banana cherry"
text2 = "apple cherry date"

In [8]:
differ = difflib.Differ()

In [12]:
list(
    differ.compare(
        text1.split(),
        text2.split()
    )
)

['  apple', '- banana', '  cherry', '+ date']

In [14]:
json1 = {"name": "John", "age": 30, "city": "New York", 
         "hobbies": ["reading", "cooking"], "isMarried": True}

json2 = {"name": "John", "age": 28, "city": "Los Angeles"}

In [15]:
import json

In [18]:
json1_str = json.dumps(json1, indent=2)
json2_str = json.dumps(json2, indent=2)

In [20]:
print(json1_str)

{
  "name": "John",
  "age": 30,
  "city": "New York",
  "hobbies": [
    "reading",
    "cooking"
  ],
  "isMarried": true
}


In [21]:
json1_str.splitlines()

['{',
 '  "name": "John",',
 '  "age": 30,',
 '  "city": "New York",',
 '  "hobbies": [',
 '    "reading",',
 '    "cooking"',
 '  ],',
 '  "isMarried": true',
 '}']

In [22]:
differ = difflib.Differ()
diff = list(
    differ.compare(
        json1_str.splitlines(),
        json2_str.splitlines(),
    )
)

In [24]:
print("\n".join(diff))

  {
    "name": "John",
-   "age": 30,
?          ^^

+   "age": 28,
?          ^^

+   "city": "Los Angeles"
-   "city": "New York",
-   "hobbies": [
-     "reading",
-     "cooking"
-   ],
-   "isMarried": true
  }


#### Easier Diffing With JsonDiff

In [32]:
!pip install jsondiff==2.0.0

  Using cached jsondiff-2.0.0-py3-none-any.whl.metadata (562 bytes)
Using cached jsondiff-2.0.0-py3-none-any.whl (6.6 kB)


In [33]:
json1 = {
    "name": "John",
    "age": 30,
    "city": "New York",
    "hobbies": ["reading", "cooking"],
    "isMarried": True,
}

json2 = {
    "age": 28,
    "name": "John",
    "city": "Los Angeles",
    "hobbies": ["cooking", "reading"],
}

In [34]:
import jsondiff

In [35]:
jsondiff.diff(json1, json2)

{'age': 28,
 'city': 'Los Angeles',
 'hobbies': {insert: [(1, 'reading')], delete: [0]},
 delete: ['isMarried']}

In [36]:
jsondiff.diff(json1, json2, syntax="explicit")

{update: {'age': 28,
  'city': 'Los Angeles',
  'hobbies': {insert: [(1, 'reading')], delete: [0]}},
 delete: ['isMarried']}

In [37]:
jsondiff.diff(json1, json2, syntax="symmetric")

{'age': [30, 28],
 'city': ['New York', 'Los Angeles'],
 'hobbies': {insert: [(1, 'reading')], delete: [(0, 'reading')]},
 delete: {'isMarried': True}}

In [38]:
# jsondiff.diff(json1, json2, syntax="compact")
diffs = jsondiff.diff(json1, json2)
diffs

{'age': 28,
 'city': 'Los Angeles',
 'hobbies': {insert: [(1, 'reading')], delete: [0]},
 delete: ['isMarried']}

In [39]:
from jsondiff import patch

In [40]:
patch(json1, diffs)

{'name': 'John',
 'age': 28,
 'city': 'Los Angeles',
 'hobbies': ['cooking', 'reading']}

In [41]:
json2

{'age': 28,
 'name': 'John',
 'city': 'Los Angeles',
 'hobbies': ['cooking', 'reading']}

In [42]:
diffs = jsondiff.diff(json2, json1)

In [43]:
patch(json2, diffs)

{'age': 30,
 'name': 'John',
 'city': 'New York',
 'hobbies': ['reading', 'cooking'],
 'isMarried': True}

In [44]:
json1

{'name': 'John',
 'age': 30,
 'city': 'New York',
 'hobbies': ['reading', 'cooking'],
 'isMarried': True}

#### DeepDiff

In [ ]:
# !pip install deepdiff==6.7.1

In [45]:
json1 = {
    "name": "John",
    "age": 30,
    "city": "New York",
    "hobbies": ["reading", "cooking"],
    "isMarried": True,
}

json2 = {
    "age": 28,
    "name": "John",
    "city": "Los Angeles",
    "hobbies": ["cooking", "reading"],
}

In [51]:
from deepdiff import DeepDiff

In [52]:
diff = DeepDiff(json1, json2)

In [53]:
diff

{'dictionary_item_removed': [root['isMarried']],
 'values_changed': {"root['age']": {'new_value': 28, 'old_value': 30},
  "root['city']": {'new_value': 'Los Angeles', 'old_value': 'New York'},
  "root['hobbies'][0]": {'new_value': 'cooking', 'old_value': 'reading'},
  "root['hobbies'][1]": {'new_value': 'reading', 'old_value': 'cooking'}}}

In [54]:
print(diff.pretty())

Item root['isMarried'] removed from dictionary.
Value of root['age'] changed from 30 to 28.
Value of root['city'] changed from "New York" to "Los Angeles".
Value of root['hobbies'][0] changed from "reading" to "cooking".
Value of root['hobbies'][1] changed from "cooking" to "reading".


In [55]:
diff = DeepDiff(json1, json2, ignore_order=True)
print(diff.pretty())

Item root['isMarried'] removed from dictionary.
Value of root['age'] changed from 30 to 28.
Value of root['city'] changed from "New York" to "Los Angeles".


In [56]:
profile1 = {
    "user": {
        "name": "Alice",
        "details": {
            "age": 30,
            "address": {"street": "123 Main St", "city": "Townsville"},
        },
    }
}

profile2 = {
    "user": {
        "name": "Alice",
        "details": {
            "age": 31,  # Age changed
            "address": {"street": "123 Main St", "city": "Villagetown"},  # City changed
        },
    }
}

In [58]:
diff = DeepDiff(profile1, profile2)
print(diff.pretty())

Value of root['user']['details']['age'] changed from 30 to 31.
Value of root['user']['details']['address']['city'] changed from "Townsville" to "Villagetown".


In [59]:
diff = DeepDiff(profile1, profile2, exclude_paths={"root['user']['details']['age']"})
print(diff.pretty())

Value of root['user']['details']['address']['city'] changed from "Townsville" to "Villagetown".


In [60]:
diff = DeepDiff(profile1, profile2, include_paths={"root['user']['details']['age']"})
print(diff.pretty())

Value of root['user']['details']['age'] changed from 30 to 31.
